<a href="https://colab.research.google.com/github/Chandradithya8/Artificial_Neural_network/blob/main/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("/content/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
a=pd.get_dummies(df[['Geography','Gender']],drop_first=True)
a.head()

,Geography_Germany,Geography_Spain,Gender_Male
0,0,0,0
1,0,1,0
2,0,0,0
3,0,0,0
4,0,1,0


In [3]:
df=df.drop(['RowNumber','CustomerId','Surname','Geography','Gender'],axis=1)
df=pd.concat([df,a],axis=1)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [4]:
x=df.drop('Exited',axis=1)
y=df['Exited']

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [6]:
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout,LeakyReLU,PReLU,ELU
from keras.activations import relu,sigmoid
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [12]:
def create_model(layers,activation):
  model=Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(units=nodes,input_dim=11))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(units=nodes))
      model.add(Activation(activation))
      model.add(Dropout(0.3))
  model.add(Dense(units=1,activation='sigmoid',kernel_initializer='glorot_uniform'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')
  return model   

In [13]:
model1=KerasClassifier(build_fn=create_model)

In [14]:
params={
    'layers':[(20), (40, 20), (45, 30, 15)], # if I use lists it shows me error
    'activation':['sigmoid','relu'],
    'batch_size':[128, 256], 
    'epochs':[30]
}
model2=GridSearchCV(estimator=model1,param_grid=params,cv=5)

In [15]:
result=model2.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


Epoch 1/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5401 - accuracy: 0.7763
Epoch 2/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5154 - accuracy: 0.7969
Epoch 3/30
50/50 [==============================] - 0s 2ms/step - loss: 0.5025 - accuracy: 0.7967
Epoch 4/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4985 - accuracy: 0.7972
Epoch 5/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4865 - accuracy: 0.7969
Epoch 6/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4731 - accuracy: 0.7962
Epoch 7/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4673 - accuracy: 0.7987
Epoch 8/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4593 - accuracy: 0.8011
Epoch 9/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4521 - accuracy: 0.8006
Epoch 10/30
50/50 [==============================] - 0s 1ms/step - loss: 0.4529 - accuracy: 0.8042
Epoch 11/30
50/50 [

In [16]:
result.best_score_

0.8583750009536744

In [17]:
result.best_params_

{'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (45, 30, 15)}